In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from pprint import pprint
import json
from scipy.stats import linregress
import scipy.stats as st
import csv
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import seaborn as sns

file_to_output = os.path.join("project-1", "output.csv")

In [2]:
#url ="https://www.fema.gov/api/open/v2/DisasterDeclarationsSummaries"
#url= "https://www.fema.gov/api/open/v1/FemaWebDisasterSummaries"
url="https://www.fema.gov/api/open/v2/HazardMitigationGrantProgramDisasterSummaries"
#filters1 = "$filter=incidentType eq 'Severe Storm'"
#filters2 = "$filter=(incidentType eq 'Hurricane' or incidentType eq 'Typhoon')" -246
#filters3 = "$filter=incidentType eq 'Fire'" -481
#filters4 = "$filter=incidentType eq 'Flood'" -355
#filters5 = "$filter=(incidentType eq 'Severe Ice Storm' or incidentType eq 'Snow Storm' or incidentType eq 'Winter Storm' or incidentType eq 'Freezing' or incidentType eq 'Coastal Storm' or incidentType eq 'Tropical Storm')"-106
#filters1 = "$filter=declarationDate gt '2014-01-13T04:00:00.000Z' and incidentType eq 'Severe Storm'" -1000
#filters1 = "$filter=declarationDate gt '2014-01-13T04:00:00.000Z' and (incidentType eq 'Hurricane' or incidentType eq 'Typhoon')"


In [3]:
# Send a GET request to the API
#response = requests.get(url+'?'+filters1)
response = requests.get(f"{url}?{filters5}")
response

<Response [200]>

In [4]:
# Send a GET request to the API


# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON data
    data = response.json()
    
    # Extract the list of disaster declarations
    #disasters = data.get('DisasterDeclarationsSummaries', [])
    #disasters = data.get('FemaWebDisasterSummaries',[])
    disasters = data.get('HazardMitigationGrantProgramDisasterSummaries',[])
    # Create a DataFrame from the list of disaster records
    Disaster_df = pd.DataFrame(disasters)
    #Disaster_df.drop(Disaster_df.columns[[8,11,12,14,15,16,17]], axis=1, inplace=True)
    #columns_to_drop = ['column1', 'column2']
    Disaster_df = Disaster_df.iloc[:, [0,1,2,3,4,5,6,8,10,13,23,24,25,26,27,28,29,31]]
    # Display the DataFrame
    Disaster_df
else:
    print(f"Error: {response.status_code}")

# Convert declarationDate to datetime
Disaster_df['declarationDate'] = pd.to_datetime(Disaster_df['declarationDate'])

# Create a new column with the desired format
Disaster_df['formattedDate'] = Disaster_df['declarationDate'].dt.strftime('%B %Y')


Disaster_df


,disasterNumber,declarationDate,disasterType,incidentType,title,region,state,hmgpCloseoutStatus,hmgpReconciliationDate,hmgpPopEndDate,obligatedTotalAmount,obligatedInitiativeAmount,obligatedPlanningAmount,obligatedRegularAmount,obligatedRecipientAdmin,obligatedSubrecipientAdmin,obligatedRecipientMgmtAmt,pendingProjectsQuantity,formattedDate
0,4244,2015-10-30 00:00:00+00:00,DR,Coastal Storm,SEVERE STORM,10,Alaska,Closed,2021-09-30T00:00:00.000Z,None,130130.29,11497.5,35275.2,54639.59,0,0,28718.00,0.0,October 2015
1,1354,2000-12-29 00:00:00+00:00,DR,Severe Ice Storm,SEVERE WINTER ICE STORM,6,Arkansas,Closed,2016-03-16T00:00:00.000Z,2007-07-31T00:00:00.000Z,28495935.00,211005.0,52006.0,27457189.00,133498,493066,149171.00,0.0,December 2000
2,4100,2013-01-29 00:00:00+00:00,DR,Severe Ice Storm,SEVERE WINTER STORM,6,Arkansas,Closed,2018-11-27T00:00:00.000Z,None,1410234.00,0.0,94000.0,1252313.00,0,0,63921.00,0.0,January 2013
3,4160,2014-01-06 00:00:00+00:00,DR,Severe Ice Storm,SEVERE WINTER STORM,6,Arkansas,Closed,2018-12-03T00:00:00.000Z,None,795042.57,0.0,55851.0,710082.84,0,0,29108.73,0.0,January 2014
4,4700,2023-04-04 00:00:00+00:00,DR,Winter Storm,SEVERE WINTER STORM,6,Arkansas,Open,None,None,0.00,0.0,0.0,0.00,0,0,0.00,NaN,April 2023
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,1681,2007-02-09 00:00:00+00:00,DR,Severe Ice Storm,SEVERE WINTER STORM,5,Illinois,Closed,2016-01-28T00:00:00.000Z,None,2866338.00,0.0,215052.0,2603088.00,0,1,48197.00,0.0,February 2007
102,851,1990-01-15 00:00:00+00:00,DR,Freezing,SEVERE FREEZE,4,Florida,Open,None,None,0.00,0.0,0.0,0.00,0,0,0.00,NaN,January 1990
103,1206,1998-03-03 00:00:00+00:00,DR,Coastal Storm,"SEVERE WINTER COASTAL STORM, HIGH WINDS, FLOODING",2,New Jersey,Closed,2002-06-05T00:00:00.000Z,None,501148.00,0.0,0.0,476631.00,10807,13710,0.00,0.0,March 1998
104,860,1990-03-06 00:00:00+00:00,DR,Severe Ice Storm,SEVERE ICE STORM,5,Illinois,Closed,1995-03-01T00:00:00.000Z,None,0.00,0.0,0.0,0.00,0,0,0.00,NaN,March 1990


In [5]:
Disaster_df['incidentType'].unique()

array(['Coastal Storm', 'Severe Ice Storm', 'Winter Storm', 'Freezing',
       'Tropical Storm'], dtype=object)

In [6]:
Disaster_df['incidentType'].value_counts()

incidentType
Severe Ice Storm    56
Coastal Storm       17
Tropical Storm      12
Winter Storm        11
Freezing            10
Name: count, dtype: int64

In [7]:
Disaster_df['title'].value_counts()

title
SEVERE WINTER STORM                                                               19
SEVERE WINTER STORMS                                                              10
SEVERE ICE STORM                                                                   9
SEVERE WINTER ICE STORM                                                            7
SEVERE WINTER STORM AND FLOODING                                                   5
SEVERE FREEZE                                                                      5
ICE STORM                                                                          4
SEVERE STORM AND FLOODING                                                          4
COASTAL STORM                                                                      3
TROPICAL STORM HELENE                                                              3
SEVERE WINTER STORMS AND FLOODING                                                  3
SEVERE WINTER STORMS AND SNOWSTORM                         